In [1]:
#uncomment and run to install textblob
#import sys
#!{sys.executable} -m pip install textblob
#!{sys.executable} -m textblob.download_corpora

In [2]:
import pandas as pd
from textblob import TextBlob

### What is the most positive sentence, and the most negative sentence in the earning calls?

### Incorporate subjectivity and select objective sentiments (low subjectivity)

In [3]:
data = pd.read_csv('data/EC10.csv')

In [4]:
data.head()

,id,text,date,company,sector
0,32934,"Zoe's Kitchen, Inc. (NYSE:ZOES)\nQ2 2015 Earni...",2015-08-28 00:11:00,"Zoe's Kitchen, Inc.",Consumer Services
1,32905,"Autodesk, Inc. (NASDAQ:ADSK)\nQ2 2016 Earnings...",2015-08-28 00:07:00,"Autodesk, Inc.",Technology
2,32926,Smith & Wesson Holding Corp. (NASDAQ:SWHC)\nQ1...,2015-08-27 23:25:00,Smith & Wesson Holding Corporation,Capital Goods
3,32930,"Ulta Salon, Cosmetics & Fragrance, Inc. (NASDA...",2015-08-27 23:09:00,"Ulta Salon, Cosmetics & Fragrance, Inc.",Consumer Services
4,32907,Splunk Inc. (NASDAQ:SPLK)\nQ2 2016 Earnings Co...,2015-08-27 22:56:00,Splunk Inc.,Technology


In [5]:
text = data.iloc[0, 1]
blob = TextBlob(text)
blob.sentiment

Sentiment(polarity=0.140799481822049, subjectivity=0.4249167174308212)

In [6]:
def max_sentiment(text):
    blob = TextBlob(text)
    mx = -1000
    ms = None
    for sent in blob.sentences:
        if mx < sent.sentiment[0]:
            mx = sent.sentiment[0]
            ms = sent.raw
    return (ms, mx)

In [7]:
data['text'].apply(max_sentiment)

0    (Aldo is truly happy when he is helping others...
1    (I'll also repeat once again that our business...
2    (We have what we believe the best margins in t...
3    (Now turning to Ultamate Rewards loyalty progr...
4    (We continue to improve the core Splunk enterp...
5    (Technology Brands grew their store count by a...
6    (Our Board of Directors unanimously concluded ...
7    (We pride ourselves in providing legendary ser...
8    (As you no doubt heard, the title was a record...
9    (So I think its best I could talk about a coup...
Name: text, dtype: object

In [8]:
for row in data['text'].apply(max_sentiment):
    print(row)

('Aldo is truly happy when he is helping others.', 0.8)
("I'll also repeat once again that our business model transition will not be perfectly linear, and the amount of business that we transition, the number of subscription additions and the mix of subscriptions issued will fluctuate from quarter-to-quarter and year-to-year.", 1.0)
('We have what we believe the best margins in the industry, so why not leverage that.', 1.0)
('Now turning to Ultamate Rewards loyalty program, we continue to reward our best guests with a differentiated experience, driving increased engagement and satisfaction.', 1.0)
('We continue to improve the core Splunk enterprise and all the core platform elements so that it can be the very best indexing engine and data analytics device known to man.', 1.0)
('Technology Brands grew their store count by an impressive 182 stores now standing at 731 stores.', 1.0)
('Our Board of Directors unanimously concluded that this arrangement is the best way to maximize value of o

In [9]:
def better_function_sentiment(text):
    #instead of a loop organize the sentences, polarity, and subjectivity values in a data frame
    blob = TextBlob(text)
    data = [ {'sentence':sent.raw, 'sentiment':sent.sentiment[0], 'subjectivity': sent.sentiment[1]}
            for sent in blob.sentences]
    return pd.DataFrame(data)

In [10]:
better_function_sentiment(text).head(10)

,sentence,sentiment,subjectivity
0,"Zoe's Kitchen, Inc. (NYSE:ZOES)\nQ2 2015 Earni...",0.35,0.45
1,Welcome to the Zoe's Kitchen Second Quarter 20...,0.40,0.45
2,"At this time, all participants have been place...",0.00,0.30
3,Please note that this conference is being reco...,0.00,0.00
4,"On the call today, we have Kevin Miles, Presid...",0.00,0.00
5,Now I'd like to turn the conference over to Ji...,0.00,0.00
6,Please go ahead.,0.00,0.00
7,"James Besch - CFO, Vice President-Accounting &...",0.70,0.60
8,"By now, everyone should have access to our sec...",0.00,0.00
9,"If not, it can be found at www.zoeskitchen.com...",0.00,0.00


In [11]:
def extract_sentences(sentiment_df, n = 5):
    data = sentiment_df.sort_values('sentiment', ascending=False).head(n)['sentence']
    return list(data)

In [12]:
extract_sentences(better_function_sentiment(text))

['Aldo is truly happy when he is helping others.',
 'Paul Westra - Stifel, Nicolaus & Co., Inc.\nGreat.',
 "And now we're happy to answer any questions you may have.",
 'Will Slabaugh - Stephens, Inc.\nGreat.',
 'Great.']

In [13]:
def extract(text):
    df = better_function_sentiment(text)
    res=extract_sentences(df)
    return res

In [14]:
#print top 5 positive sentiment sentences in each earning call
for row in data['text'].apply(extract):
    print(row)

['Aldo is truly happy when he is helping others.', 'Paul Westra - Stifel, Nicolaus & Co., Inc.\nGreat.', "And now we're happy to answer any questions you may have.", 'Will Slabaugh - Stephens, Inc.\nGreat.', 'Great.']
['Perfect.', 'Keith Eric Weiss - Morgan Stanley & Co. LLC\nExcellent.', "I'll also repeat once again that our business model transition will not be perfectly linear, and the amount of business that we transition, the number of subscription additions and the mix of subscriptions issued will fluctuate from quarter-to-quarter and year-to-year.", 'Excellent.', "You're welcome."]
['We have what we believe the best margins in the industry, so why not leverage that.', 'Have a great day.', 'I want to thank everyone at Smith & Wesson for delivering a great quarter.', 'Great quarter.', "King & Associates, Inc.\nThat's great."]
['Best regards.', 'Best of luck for the rest of the year.', 'Best of luck.', 'Best of luck.', 'Now turning to Ultamate Rewards loyalty program, we continue t

### Incorporate subjectivity and select objective sentiments (low subjectivity)

In [15]:
def extract_sentences2(sentiment_df, n = 5):
    #instead of using sentiment, multiply by 1-sujecivity
    sentiment_df['ns'] = sentiment_df['sentiment'] * (1 - sentiment_df['subjectivity'])
    data = sentiment_df.sort_values('ns', ascending=False).head(n)['sentence']
    return list(data)

In [16]:
def extract2(text):
    df = better_function_sentiment(text)
    res=extract_sentences2(df)
    return res

In [17]:
for row in data['text'].apply(extract2):
    print(row)

["Also during today's call, we will discuss non-GAAP measures, which we believe can be useful in evaluating our performance.", 'Good afternoon, everyone, and thank you for joining us on the call today.', 'We have a good – a backfill strategy going.', 'Good stuff.', 'Good afternoon.']
['Good afternoon.', 'Thank you very much, and good afternoon, guys.', "We told you at the time, some of these were our best guess and mapped most easily to the history before it, and we said as we learned more about it and understood it better and would better pinpoint the things that we're looking at.", 'Okay.', "And so that's the point of providing that and we'll spend more time talking about this...\nHeather Anne Bellini - Goldman Sachs & Co.\nNo, no."]
['We have what we believe the best margins in the industry, so why not leverage that.', "It's more of an analysis of our best use of cash at any particular given time.", 'This quarter was a good quarter and exceeded our expectations.', "So we think there